In [ ]:
# ==============================================
# Crop Disease Detection using Deep Learning
# Using MobileNet and InceptionV3 (Transfer Learning)
# ==============================================

# 1. Imports and Setup
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Setting seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# 2. Leaf Segmentation and Preprocessing Function
def preprocess_input_image(img):
    # Convert to float32
    img = img.astype("float32") / 255.0

    # Resize to 224x224
    img = cv2.resize(img, (224, 224))

    return img



# 3. Data Paths (placeholders for PlantVillage dataset)
# Stage 1 (Crop Type) data directory (each subfolder is a crop type)
crop_data_dir = '/content/drive/MyDrive/Dataset/crop'
# Stage 2 (Disease) data directory: each subfolder is a crop, containing disease subfolders
disease_data_dir = '/content/drive/MyDrive/Dataset/disease'

# 4. ImageDataGenerator Setup (augmentation + preprocessing)
train_datagen = ImageDataGenerator(
    # rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,               # 80-20 split
    preprocessing_function=preprocess_input_image
)

# 5. Stage 1: Crop Classification Model (MobileNetV2)
train_generator = train_datagen.flow_from_directory(
    crop_data_dir,
    target_size=(224, 224),            # initial size for segmentation (cropping to 224 later)
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)
val_generator = train_datagen.flow_from_directory(
    crop_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)
num_crops = train_generator.num_classes  # number of crop classes

# Building MobileNetV2 model for crop classification
base_model_crop = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model_crop.layers:
    layer.trainable = False
x = base_model_crop.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # dropout to avoid overfitting
output_crops = Dense(num_crops, activation='softmax')(x)
model_crop = Model(inputs=base_model_crop.input, outputs=output_crops)

model_crop.compile(optimizer=Adam(),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

# Train the crop classification model
history_crop = model_crop.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Plot of training & validation accuracy and loss for crop model
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_crop.history['accuracy'], label='Train Accuracy')
plt.plot(history_crop.history['val_accuracy'], label='Val Accuracy')
plt.title('Crop Classification Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history_crop.history['loss'], label='Train Loss')
plt.plot(history_crop.history['val_loss'], label='Val Loss')
plt.title('Crop Classification Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 6. Stage 2: Disease Classification Models (InceptionV3 per Crop)
disease_datagen = ImageDataGenerator(
    # rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    preprocessing_function=preprocess_input_image
)

# Get list of crop types from the crop generator
crop_names = list(train_generator.class_indices.keys())

for crop in crop_names:
    print(f"\n--- Training disease model for crop: {crop} ---")
    # Paths to this crop's disease images
    crop_disease_path = os.path.join(disease_data_dir, crop)

    train_gen = disease_datagen.flow_from_directory(
        crop_disease_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )
    val_gen = disease_datagen.flow_from_directory(
        crop_disease_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='validation',
        shuffle=True
    )
    num_diseases = train_gen.num_classes

    # Build InceptionV3 model for disease classification (crop-specific)
    base_model_disease = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model_disease.layers:
        layer.trainable = False
    y = base_model_disease.output
    y = GlobalAveragePooling2D()(y)
    y = Dropout(0.5)(y)
    output_diseases = Dense(num_diseases, activation='softmax')(y)
    model_disease = Model(inputs=base_model_disease.input, outputs=output_diseases)

    model_disease.compile(optimizer=Adam(),
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])

    # Train the disease classification model for this crop
    history_disease = model_disease.fit(
        train_gen,
        epochs=10,
        validation_data=val_gen
    )

    # Plot training & validation accuracy and loss for this disease model
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history_disease.history['accuracy'], label='Train Accuracy')
    plt.plot(history_disease.history['val_accuracy'], label='Val Accuracy')
    plt.title(f'{crop.capitalize()} Disease Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history_disease.history['loss'], label='Train Loss')
    plt.plot(history_disease.history['val_loss'], label='Val Loss')
    plt.title(f'{crop.capitalize()} Disease Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


In [ ]:
#Saving models 
model_crop.save("/content/drive/MyDrive/Dataset/models/crop_model.h5")
model_disease.save(f"/content/drive/MyDrive/Dataset/models/{crop}_disease_model.h5")


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

crop_model = load_model("/content/drive/MyDrive/Dataset/models/crop_model.h5")

disease_models = {}
for crop in ["Potato","Rice","Cashew","Potato","Tomato"]:
    path = f"/content/drive/MyDrive/Dataset/models/{crop}_disease_model.h5"
    disease_models[crop] = load_model(path)

def preprocess_input_image(img):
    img = cv2.resize(img, (224, 224))
    img = img.astype("float32") / 255.0
    return img

def predict_crop_and_disease(image_path):
    # Read image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Preprocess
    processed = preprocess_input_image(img)
    processed = np.expand_dims(processed, axis=0)

    # 1️⃣ Predict crop
    crop_pred = crop_model.predict(processed)[0]
    crop_index = np.argmax(crop_pred)
    crop_label = list(train_generator.class_indices.keys())[crop_index]
    print("Predicted Crop:", crop_label)

    # 2️⃣ Predict disease using its model
    disease_model = disease_models[crop_label]
    disease_pred = disease_model.predict(processed)[0]
    disease_index = np.argmax(disease_pred)

    # Get disease labels
    disease_path = f"/content/drive/MyDrive/Dataset/disease/{crop_label}"
    disease_classes = sorted(os.listdir(disease_path))

    disease_label = disease_classes[disease_index]
    print("Predicted Disease:", disease_label)

    return crop_label, disease_label

#Testing model by providing a leaf image of potato
predict_crop_and_disease("/content/healthy_potato_leaf.jpeg")
